# Rule-based

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 15:44:56--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  64.4MB/s    in 0.2s    

2021-12-26 15:44:56 (64.4 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-26 15:44:56--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  56.6MB/s    in 0.2s    

202

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [20]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    code
    '''
    # 近一月熱銷商品
    recent_data = training_data[(training_data['DATE'] >= '2018-08-01') & (training_data['DATE'] < '2018-09-01')]
    recent_top = recent_data.groupby('asin') \
       .agg({'asin':'size', 'overall':'mean'}) \
       .rename(columns={'asin':'sold_count','overall':'mean_score'}) \
       .reset_index().sort_values('sold_count', ascending=False) \
       .head(k).asin.reset_index(drop=True).to_list()
    # 找出每個客戶高評價的商品
    high_ratings = ratings_trainings.loc[ratings_trainings['overall'] >= 3]
    user_high_ratings = high_ratings.groupby(['reviewerID','asin'])['asin'] \
        .agg(['count']) \
        .reset_index()
    # 找出每個商品相關商品
    metadata['similar'] = [x+y for x, y in zip(metadata.also_buy, metadata.also_view)]
    part_metadata = metadata[["asin", "similar"]]
    # 建立推薦
    for user in users:
      tmp_recommendations = recent_top
      if (user in user_high_ratings['reviewerID'].tolist()):
        user_similar_products = part_metadata.loc[part_metadata['asin'] == user_high_ratings.loc[user_high_ratings['reviewerID'] == user]['asin'].tolist()[0]]['similar'].tolist()
        if len(user_similar_products) > 0:
          if len(user_similar_products[0]) > k:
            tmp_recommendations = user_similar_products[0][:k]
          else:
            tmp_recommendations = user_similar_products[0] + recent_top[:(k-len(user_similar_products[0]))]
      recommendations[user] = tmp_recommendations
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01DLR9IDI',
  'B01AVJCDYA',
  'B0195R1FT8',
  'B01CJNZKZK',
  'B018WCT01C',
  'B0168SXRR0'],
 'A103T1QOGFCSEH': ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01DLR9IDI',
  'B01AVJCDYA',
  'B0195R1FT8',
  'B01CJNZKZK',
  'B018WCT01C',
  'B0168SXRR0'],
 'A106UKKSJ2KXPF': ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01DLR9IDI',
  'B01AVJCDYA',
  'B0195R1FT8',
  'B01CJNZKZK',
  'B018WCT01C',
  'B0168SXRR0'],
 'A10A7GV4D5A11V': ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01DLR9IDI',
  'B01AVJCDYA',
  'B0195R1FT8',
  'B01CJNZKZK',
  'B018WCT01C',
  'B0168SXRR0'],
 'A1119JJ37ZLB8R': ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01DLR9IDI',
  'B01AVJCDYA',
  'B0195R1FT8',
  'B01CJNZKZK',
  'B018WCT01C',
  'B0168SXRR0'],
 'A113UOOLBSZN52': ['B01DKQAXC0',
  'B00W259T7G',
  'B013XKHA4M',
  'B012Z7IHHI',
  'B01DLR9IDI',
  'B01A

## 結果評估

In [21]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.15254237288135594